# Project: Youtube Analyzer
## Authors: Karmehr Arora, Alfred Koh

### Project Overview
Data: Youtube API’s
GUI: Tkinter

Roles:

    Karmehr: Accessing Data from the API & organizing it

    Alfred: Analyzing the data & representing it with tkinter

Goal: 
Finding the top 10 trending basketball videos (or videos of a requested topic) in the last 24 hours
Look for the most (total likes + total comments + total shares) under tag = basketball in the last 24 hours

Limitations: 
Estimated limit of searches of 50 per day due to Youtube API usage restrictions


In [1]:
# import utilized libraries pertaining to data collection
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [2]:
# get access key to API
# don't push to github, but hardcode for demo
from getpass import getpass
API_KEY = getpass("Enter your API Key: ")

Enter your API Key: ········


In [3]:
# define the api service and version
api_service_name = "youtube"
api_version = "v3"

## Requesting User to choose topic
default to basketball as topic if user has error/opts out of entering topic

In [4]:
# request user for topic (default = 'basketball')

# replace topic = input() with GUI that prompts user for topic to search for
topic = input()

if(len(topic) == 0):
    topic = "basketball"

## Retrieving current Datetime

In [14]:
from datetime import datetime
from datetime import timedelta

# datetime object containing current date and time
current_datetime = datetime.now() - timedelta(days=7)
print("datetime =", current_datetime)

# in accordance with Youtube Datetime rules; format =  YY/mm/ddTH:M:SZ
modified_dt = current_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")
print("date and time =", modified_dt)

datetime = 2023-11-24 23:24:14.159636
date and time = 2023-11-24T23:24:14Z


## Search youtube for videos with that tag

In [19]:
# access the youtube api client
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = API_KEY)

# make a request to the api to find videos under that topic
# note: cost of this operation is 100 units of the alloted 10,000 units per 24 hours
request = youtube.search().list(
        part="id,snippet",
        maxResults=50,
        publishedAfter=modified_dt,
        q=topic
    )

In [20]:
# execute the query in the request
response = request.execute()

In [23]:
print("Search ID: ", response["items"][0]['id'])
print("\nSnippet Data:",response["items"][0]['snippet'], '\n')
videoIDs = []
videos = response["items"]
count = 1
for video in videos:
    if('videoId' in video['id']):
        print(count, ": https://www.youtube.com/watch?v=" + video['id']['videoId'])
        videoIDs.append(video['id']['videoId'])
        print(video['snippet']['publishedAt'])
        count = count+1
    else: 
        # this seems to only print out channels meaning the youtube search only returns videos & channels
        print(video['snippet'])

Search ID:  {'kind': 'youtube#video', 'videoId': 'x2FD8gmA5-w'}

Snippet Data: {'publishedAt': '2023-12-01T05:45:27Z', 'channelId': 'UCWJ2lWNubArHWmf3FIHbfcQ', 'title': 'CLIPPERS at WARRIORS | FULL GAME HIGHLIGHTS | November 30, 2023', 'description': 'Never miss a moment with the latest news, trending stories and highlights to bring you closer to your favorite players and teams.', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/x2FD8gmA5-w/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/x2FD8gmA5-w/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/x2FD8gmA5-w/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'NBA', 'liveBroadcastContent': 'none', 'publishTime': '2023-12-01T05:45:27Z'} 

1 : https://www.youtube.com/watch?v=x2FD8gmA5-w
2023-12-01T05:45:27Z
2 : https://www.youtube.com/watch?v=7k109F-4Hbo
2023-12-02T03:03:21Z
3 : https://www.youtube.com/watch?v=VUkm6b58bMo
2023-12-01T15:09:25Z
4

## Calculate and Find top 10 most trending videos under that tag

In [10]:
collection_of_video_data = {}
for video in videos:
    if('videoId' in video['id']):
        request = youtube.videos().list(part = 'snippet, statistics', id = video['id']['videoId'])
        response = request.execute()
        collection_of_video_data[video['id']['videoId']] = response

In [38]:
set_of_trend_totals = []
count = 1
for vid_data in collection_of_video_data:
#     print(count, ':', collection_of_video_data[vid_data])
    print("Statistics: ", collection_of_video_data[vid_data]['items'][0]['statistics'])
    
    trendTotal = 0
    if "viewCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        views = collection_of_video_data[vid_data]['items'][0]['statistics']['viewCount']
        trendTotal = trendTotal + int(views)
    if "likeCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        likes = collection_of_video_data[vid_data]['items'][0]['statistics']['likeCount']
        trendTotal = trendTotal + int(likes)
    if "commentCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        comments = collection_of_video_data[vid_data]['items'][0]['statistics']['commentCount']
        trendTotal = trendTotal + int(comments)

    set_of_trend_totals.append(trendTotal)
    print(count, ': Trend Total: ', trendTotal)
    count = count+1

Statistics:  {'viewCount': '1096364', 'likeCount': '11884', 'favoriteCount': '0', 'commentCount': '999'}
1 : Trend Total:  1109247
Statistics:  {'viewCount': '116568', 'likeCount': '1428', 'favoriteCount': '0', 'commentCount': '209'}
2 : Trend Total:  118205
Statistics:  {'viewCount': '126276', 'likeCount': '3026', 'favoriteCount': '0', 'commentCount': '152'}
3 : Trend Total:  129454
Statistics:  {'viewCount': '522953', 'likeCount': '4818', 'favoriteCount': '0', 'commentCount': '1100'}
4 : Trend Total:  528871
Statistics:  {'viewCount': '2273917', 'likeCount': '38186', 'favoriteCount': '0', 'commentCount': '2615'}
5 : Trend Total:  2314718
Statistics:  {'viewCount': '555896', 'likeCount': '5178', 'favoriteCount': '0', 'commentCount': '614'}
6 : Trend Total:  561688
Statistics:  {'viewCount': '273712', 'likeCount': '3990', 'favoriteCount': '0', 'commentCount': '504'}
7 : Trend Total:  278206
Statistics:  {'viewCount': '1188814', 'likeCount': '15119', 'favoriteCount': '0', 'commentCount'

In [12]:
count = 0
video_id_to_trend_total = {}
for video in videoIDs:
    print('videoID: trendTotal --> ', video, end = ": ")
    print(set_of_trend_totals[count])
    video_id_to_trend_total[video] = set_of_trend_totals[count]
    count = count + 1
    
# for video in video_id_to_trend_total:
#     print(video, ":", video_id_to_trend_total[video])

videoID: trendTotal -->  x2FD8gmA5-w: 1109247
videoID: trendTotal -->  7k109F-4Hbo: 118205
videoID: trendTotal -->  VUkm6b58bMo: 129454
videoID: trendTotal -->  oCfSiZ0lFio: 528871
videoID: trendTotal -->  0LpWxdca-dw: 2314718
videoID: trendTotal -->  sAUL0EqcBB4: 561688
videoID: trendTotal -->  wkl7d21nYEU: 278206
videoID: trendTotal -->  CloeoWpNzmE: 1204669
videoID: trendTotal -->  HeH_x5dSC9A: 234150
videoID: trendTotal -->  9qRl8rNwehs: 427053
videoID: trendTotal -->  ldIDsqN8A0A: 10459
videoID: trendTotal -->  ZoEvDS5EPQ8: 36793
videoID: trendTotal -->  Xi9_BIkOcxc: 290985
videoID: trendTotal -->  zZWsBf2aK9Q: 203549
videoID: trendTotal -->  POTWnhp4lnc: 76836
videoID: trendTotal -->  1HJ66GoCD48: 793222
videoID: trendTotal -->  X6hUWLlPOlM: 189493
videoID: trendTotal -->  b10Mou6svz0: 115372
videoID: trendTotal -->  cNHfr-x7Xnw: 112491
videoID: trendTotal -->  QWMjsAv3ZbM: 104888
videoID: trendTotal -->  OJWFg0EisNE: 531747
videoID: trendTotal -->  mqoad2zBNxQ: 502931
videoID: t

In [39]:
# this is a collection of thumbnails for tkinter
for video in videos:
    print("Video Thumbails for tkinter:",video['snippet']['thumbnails']['medium']['url'])

Video Thumbails for tkinter: https://i.ytimg.com/vi/x2FD8gmA5-w/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/7k109F-4Hbo/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/VUkm6b58bMo/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/oCfSiZ0lFio/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/0LpWxdca-dw/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/sAUL0EqcBB4/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/wkl7d21nYEU/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/CloeoWpNzmE/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/HeH_x5dSC9A/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/9qRl8rNwehs/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/ldIDsqN8A0A/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/Xi9_BIkOcxc/mqdefault.jpg
Video Thumbails for tkinter: https://i.ytimg.com/vi/zZWsBf2aK9Q/